In [220]:
#META DATA - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - 

    # Developer details: 
        # Name: Harish S
        # Role: Architect
        # Code ownership rights: Harish S
    # Version:
        # Version: V 1.0 (Sept 16 )
            # Developer: Harish S
     
    # Description: The code enables to explore features of Feature Engine library
    
# CODE - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - 

# Dependency: 
    # Environment:     
        #Python 3.12

Source of the data: https://archive.ics.uci.edu/dataset/20/census+income


Import the libraries

In [182]:
import numpy as np #import numpy for numerical computation
import pandas as pd #import pandas for data manipulation
import feature_engine #import feature engine for feature preprocessing 

Source of the data : 

In [183]:
#1.pip install ucimlrepo 

In [184]:
#2. Run the below code to download the data

In [185]:
# fetch datasets from the ucimlrepo
from ucimlrepo import fetch_ucirepo 

# fetch dataset 
census_income = fetch_ucirepo(id=20) 
  
# data (as pandas dataframes) 
df_features = census_income.data.features 
df_target = census_income.data.targets

Visualise the data

In [186]:
df_features.head() #visualise the features 

,age,workclass,fnlwgt,education,education-num,marital-status,occupation,relationship,race,sex,capital-gain,capital-loss,hours-per-week,native-country
0,39,State-gov,77516,Bachelors,13,Never-married,Adm-clerical,Not-in-family,White,Male,2174,0,40,United-States
1,50,Self-emp-not-inc,83311,Bachelors,13,Married-civ-spouse,Exec-managerial,Husband,White,Male,0,0,13,United-States
2,38,Private,215646,HS-grad,9,Divorced,Handlers-cleaners,Not-in-family,White,Male,0,0,40,United-States
3,53,Private,234721,11th,7,Married-civ-spouse,Handlers-cleaners,Husband,Black,Male,0,0,40,United-States
4,28,Private,338409,Bachelors,13,Married-civ-spouse,Prof-specialty,Wife,Black,Female,0,0,40,Cuba


In [187]:
df_features.info() #check the dtypes and shape

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 48842 entries, 0 to 48841
Data columns (total 14 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   age             48842 non-null  int64 
 1   workclass       47879 non-null  object
 2   fnlwgt          48842 non-null  int64 
 3   education       48842 non-null  object
 4   education-num   48842 non-null  int64 
 5   marital-status  48842 non-null  object
 6   occupation      47876 non-null  object
 7   relationship    48842 non-null  object
 8   race            48842 non-null  object
 9   sex             48842 non-null  object
 10  capital-gain    48842 non-null  int64 
 11  capital-loss    48842 non-null  int64 
 12  hours-per-week  48842 non-null  int64 
 13  native-country  48568 non-null  object
dtypes: int64(6), object(8)
memory usage: 5.2+ MB


Missing values

In [188]:
df_features.isnull().sum() #check the no of missing values

age                 0
workclass         963
fnlwgt              0
education           0
education-num       0
marital-status      0
occupation        966
relationship        0
race                0
sex                 0
capital-gain        0
capital-loss        0
hours-per-week      0
native-country    274
dtype: int64

we see that the object type features "workclass", "occupation" and "native country" have missing values , hence apply categorical imputation from feature engine library

In [189]:
from feature_engine.imputation import CategoricalImputer #import the imputer

In [190]:
ci = CategoricalImputer(imputation_method='frequent') # initiate the imputer

In [191]:
df_features=ci.fit_transform(df_features) #transform the feature to apply imputation

In [192]:
df_features.isnull().sum() #recheck the imputation

age               0
workclass         0
fnlwgt            0
education         0
education-num     0
marital-status    0
occupation        0
relationship      0
race              0
sex               0
capital-gain      0
capital-loss      0
hours-per-week    0
native-country    0
dtype: int64

In [193]:
cat_cols=df_features.select_dtypes(include=['object']).columns.to_list() # select the categorical columns

In [194]:
df_cat=df_features[cat_cols] # subset the categorical data

In [195]:
df_cat #visualise the categorical dataframe 

,workclass,education,marital-status,occupation,relationship,race,sex,native-country
0,State-gov,Bachelors,Never-married,Adm-clerical,Not-in-family,White,Male,United-States
1,Self-emp-not-inc,Bachelors,Married-civ-spouse,Exec-managerial,Husband,White,Male,United-States
2,Private,HS-grad,Divorced,Handlers-cleaners,Not-in-family,White,Male,United-States
3,Private,11th,Married-civ-spouse,Handlers-cleaners,Husband,Black,Male,United-States
4,Private,Bachelors,Married-civ-spouse,Prof-specialty,Wife,Black,Female,Cuba
...,...,...,...,...,...,...,...,...
48837,Private,Bachelors,Divorced,Prof-specialty,Not-in-family,White,Female,United-States
48838,Private,HS-grad,Widowed,Prof-specialty,Other-relative,Black,Male,United-States
48839,Private,Bachelors,Married-civ-spouse,Prof-specialty,Husband,White,Male,United-States
48840,Private,Bachelors,Divorced,Adm-clerical,Own-child,Asian-Pac-Islander,Male,United-States


In [196]:
df_category=df_cat.copy(deep=True) #copying the categorical dataframe to new dataframe df_category for further processing

Categorical to numerical conversion

In [197]:
from feature_engine.encoding import OrdinalEncoder # import encoder

In [198]:
od = OrdinalEncoder(encoding_method='arbitrary') # encode the categorical values 

In [199]:
df_category=od.fit_transform(df_category) #transform the categories into numbers

In [200]:
df_category # visualise the transformed values

,workclass,education,marital-status,occupation,relationship,race,sex,native-country
0,0,0,0,0,0,0,0,0
1,1,0,1,1,1,0,0,0
2,2,1,2,2,0,0,0,0
3,2,2,1,2,1,1,0,0
4,2,0,1,3,2,1,1,1
...,...,...,...,...,...,...,...,...
48837,2,0,2,3,0,0,1,0
48838,2,1,6,3,5,1,0,0
48839,2,0,1,3,1,0,0,0
48840,2,0,2,0,3,2,0,0


we can see that the cateorical column has been converted to numbers

Target Distribution

In [201]:
df_target.head() #visualise the target values

,income
0,<=50K
1,<=50K
2,<=50K
3,<=50K
4,<=50K


In [202]:
df_target.value_counts() #to display the number of categories

income
<=50K     24720
<=50K.    12435
>50K       7841
>50K.      3846
Name: count, dtype: int64

MAP the target into two categories

In [203]:
df_target.loc[:,'income']=df_target['income'].map({"<=50K":0,"<=50K.":0,">50K":1,">50K.":1}) # the duplicates will be consilidated and mapped as one category

In [204]:
df_target.value_counts() #to display the number of categories

income
0         37155
1         11687
Name: count, dtype: int64

In [205]:
df_target=df_target.astype("category") # for the ML model to recognise a binary classification problem

Remove the original categorical columns and replace it with ordinal encoded values in the features dataframe

In [206]:
df_features=df_features.drop(columns=cat_cols)

In [207]:
df_features.head() #categorical columns would be removed

,age,fnlwgt,education-num,capital-gain,capital-loss,hours-per-week
0,39,77516,13,2174,0,40
1,50,83311,13,0,0,13
2,38,215646,9,0,0,40
3,53,234721,7,0,0,40
4,28,338409,13,0,0,40


In [208]:
df_final=pd.concat([df_features,df_category],axis=1) #join the original dataframe with categorical values

In [209]:
df_final #check if two dataframes are joined

,age,fnlwgt,education-num,capital-gain,capital-loss,hours-per-week,workclass,education,marital-status,occupation,relationship,race,sex,native-country
0,39,77516,13,2174,0,40,0,0,0,0,0,0,0,0
1,50,83311,13,0,0,13,1,0,1,1,1,0,0,0
2,38,215646,9,0,0,40,2,1,2,2,0,0,0,0
3,53,234721,7,0,0,40,2,2,1,2,1,1,0,0
4,28,338409,13,0,0,40,2,0,1,3,2,1,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
48837,39,215419,13,0,0,36,2,0,2,3,0,0,1,0
48838,64,321403,9,0,0,40,2,1,6,3,5,1,0,0
48839,38,374983,13,0,0,50,2,0,1,3,1,0,0,0
48840,44,83891,13,5455,0,40,2,0,2,0,3,2,0,0


Now we have transformed features in df_final and transformed target data in df_target for ML modelling

In [210]:
# Split the data for ML modelling

from sklearn.model_selection import train_test_split #import train test split class

xtrain,xtest,ytrain,ytest = train_test_split(df_final,df_target,test_size=0.2) # split into train and test 

In [211]:
#Flatten the ytrain values
Ytrain=ytrain.values.ravel()

In [212]:
#Flatten the ytest values
Ytest=ytest.values.ravel()

Apply ML model

In [213]:
from sklearn.linear_model import LogisticRegression #import Logistic regression class 

In [214]:
LRcl=LogisticRegression(max_iter=1000,solver='liblinear') #use liblinear and max iter of 1000 to converge 

In [215]:
LRcl.fit(xtrain,Ytrain) # fit the model on train data 

LogisticRegression(max_iter=1000, solver='liblinear')

In [216]:
ypred = LRcl.predict(xtest) #predict on a test dataframe

METRICS

In [217]:
from sklearn.metrics import classification_report,precision_score,recall_score # import the metrics

In [218]:
print("precision_score: ",precision_score(Ytest,ypred))
print("recall_score: ",recall_score(Ytest,ypred))

precision_score:  0.7203791469194313
recall_score:  0.26457789382071367


In [219]:
print("classification report: ",classification_report(Ytest,ypred))

classification report:                precision    recall  f1-score   support

           0       0.81      0.97      0.88      7471
           1       0.72      0.26      0.39      2298

    accuracy                           0.80      9769
   macro avg       0.77      0.62      0.63      9769
weighted avg       0.79      0.80      0.77      9769



Inferences :

Using feature engine library one can apply transformation such as imputation, encoding etc depending on the transformation required on the data